### Model Call

In [35]:
# Google Drive Mounting

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
# 'base' variable has to store the path to the folder where the model, test set and validation set of BanglaRQA is saved/available/uploaded/stored
# so set the 'base' variable path to the folder of the where you have uploaded/saved the model, test set and validation set
base = '/content/drive/MyDrive/BQA'       #sample

# 'test_file_name' variable has to store file name of the test set of BanglaRQA
# so set the 'test_file_name' variable as the file anme of the test set of BanglaRQA
test_file_name = 'Test.json' #sample

# 'validation_file_name' variable has to store file name of the validation set of BanglaRQA
# so set the 'validation_file_name' variable as the file anme of the validation set of BanglaRQA
validation_file_name = 'Validation.json'      #sample


# 'model_name' variable has to store file name of the saved/uploaded/stroed model that you want to test
# so set the 'model_name' variable as the file anme of saved/uploaded/stroed model you want to test
#model_name = 'model_weights_epoch_10.pth'         #sample
model_name = 'mBERT_model_weights_epoch_14.pth'

In [37]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
pip install git+https://github.com/csebuetnlp/normalizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-mk9o7_a3
  Running command git clone -q https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-mk9o7_a3


In [39]:
import torch

In [40]:
# calling the model BanglaT5 to use it as a class

from transformers import AutoModelForTokenClassification, AutoTokenizer
from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer

model = AutoModelForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased", use_fast=True,)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [41]:
model.config.architectures = ['BertForTokenClassification']

In [42]:
from transformers import AdamW

# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# move model over to detected device

optimizer = AdamW(model.parameters(), lr=5e-5)


#calling the checkpoint and loading the parameters of the saved model for evaluation
checkpoint = torch.load(base + '/'+ model_name)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.to(device)


model.eval()
print('done')

done


In [43]:
from collections import Counter

# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    #'''
    common_tokens = Counter(pred_tokens) & Counter(truth_tokens)
    common_tokens = sum(common_tokens.values())
    #'''

    '''
    common_tokens = set(pred_tokens) & set(truth_tokens)
    common_tokens = len(common_tokens)
    '''
    
    # if there are no common tokens then f1 = 0
    if common_tokens == 0:
        return 0
    
    prec = 1.0 * common_tokens / len(pred_tokens)
    rec = 1.0 * common_tokens / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)


In [44]:
import numpy as np
from scipy.optimize import linear_sum_assignment

def get_updated_f1(prediction, truth):
  # first e prediction k ; diye bhag korte hobe
  # truth keo ; diye bhag korte hobe
  splitted_prediction = prediction.split(';')
  #print(splitted_prediction)
  splitted_truth = truth.split(';')
  #print(splitted_truth)

  # most probably pair wise compute_f1 chalaite hobe
  scores = np.zeros([len(splitted_truth), len(splitted_prediction)])
  for gold_index, gold_item in enumerate(splitted_truth):
    for pred_index, pred_item in enumerate(splitted_prediction):
      scores[gold_index, pred_index] = compute_f1(pred_item, gold_item)

  #print(scores)

  row_ind, col_ind = linear_sum_assignment(-scores)

  max_scores = np.zeros([max(len(splitted_truth), len(splitted_prediction))])
  for row, column in zip(row_ind, col_ind):
    max_scores[row] = max(max_scores[row], scores[row, column])
  
  #print(max_scores)

  # align korte hobe kemne jani na

  # protita prediction er average nite hobe
  # average ta return korte hobe
  f1 = np.mean(max_scores)
  return f1

#get_updated_f1("aaaa qqqq; sdfs fsf; fssfsfs", "aaaa; fsr qqqq; ooooo; a");

In [45]:
labels_to_ids = {k: v for v, k in enumerate(['B', 'I', 'O'])}
ids_to_labels = {v: k for v, k in enumerate(['B', 'I', 'O'])}
print(labels_to_ids)
print(ids_to_labels)

{'B': 0, 'I': 1, 'O': 2}
{0: 'B', 1: 'I', 2: 'O'}


In [46]:
label_all_tokens = False

def align_word_ids(qc):
  
    tokenized_inputs = tokenizer(qc, max_length=512, padding="max_length", truncation=True, return_attention_mask=True, add_special_tokens=False, return_tensors="pt")
    #print(qc)
    word_ids = tokenized_inputs.word_ids()
    #print(word_ids)
    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(1)
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(1 if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids


def evaluate_one_text(qc):

    text = tokenizer(qc, max_length=512, padding="max_length", truncation=True, return_attention_mask=True, add_special_tokens=False, return_tensors="pt")

    mask = text['attention_mask'].to(device)
    input_id = text['input_ids'].to(device)
    label_ids = torch.Tensor(align_word_ids(qc)).unsqueeze(0).to(device)

    logits = model(input_id, mask, None)
    #print(logits)
    #print(len(logits[0][0]))
    #logits_clean = logits[0][label_ids != -100]

    predictions = logits[0][0].argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]

    #print(input_id)
    a = tokenizer.convert_ids_to_tokens(input_id[0])

    '''
    print(prediction_label)
    print(len(prediction_label))
    print(a)
    print(len(a))
    '''

    ans = ''
    ind = True
    for i in range(len(prediction_label)):
      #print(i)
      if(prediction_label[i] == 'B' and ind == True):
        ans = ans + a[i]
        ind = False
      elif (prediction_label[i] == 'B'):
        ans = ans + '; ' + a[i]
      elif (label_ids[0][i] == -100 and prediction_label[i] == 'I'):
        ans = ans + a[i]
      elif (prediction_label[i] == 'I'):
        ans = ans + ' ' + a[i]

    return ans

In [47]:
import string, re
def remove_punc(text):
  exclude = set(string.punctuation)
  exclude.remove('?')
  return "".join(ch for ch in text if ch not in exclude)

### BanglaRQA Validation

In [48]:
# loading the validation dataset for calculating the EM and F1 scores on BanglaRQA's validation set

import json
import os

f = open(os.path.join(base,validation_file_name))
  
data_val = json.load(f)
  
data_val.keys()
  
data_val = data_val['data']

In [49]:
context_val = []
question_val = []
answer_val = []
answer_type_val = []

for i in range(len(data_val)):
    for j in range(len(data_val[i]['qas'])):
        context_val.append(remove_punc(normalize(data_val[i]['context'])))
        question_val.append(remove_punc(normalize(data_val[i]['qas'][j]['question_text'])))
        answer_val.append(normalize(data_val[i]['qas'][j]['answers']['answer_text'][0]))
        answer_type_val.append(normalize(data_val[i]['qas'][j]['answers']['answer_type'][0]))

In [50]:
print(len(context_val))
print(context_val[0])

for i in range(len(context_val)):
  context_val[i] = context_val[i].replace('।', ' ।')
  answer_val[i] = answer_val[i].replace('।', ' ।')
  question_val[i] = question_val[i].replace('।', ' ।')

print(context_val[0])

1484
আস্যাইরিয়ান ও ব্যাবলিয়ান সাহিত্যে সরগন তার অবনমিত অবস্থান থেকে ক্ষমতায় উত্থান ও মেসোপটেমিয়া অভিযানের জন্য কিংবদন্তি গল্পের মুল উপজীব্যে পরিনত হয়েছিলেন এরকম আরও কিছু কিছু আংশিক কিংবদন্তী উপাখ্যান ছাড়াও সারগনের খোদ নিজের অনেক লিপি আছে যদিও তার বেশিরভাগ পরবর্তী সংস্করনগুলি থেকে নেওয়া। ল্যুভর জাদুঘরে দুটি সারগনিক বিজয় ফলক এর অংশবিশেষ আছে যা সুসা যেখানে এগুলি খুব সম্ভবত মেসোপটেমিয়া ত্থেকে দ্বাদশ শতাব্দীতে স্থানান্তরিত করা হয়েছিল থেকে পুনঃউদ্ধার করা হয়েছিল। দৃশ্যত সরগন সেমেটিক আক্কাদীয়ান ভাষার লিপির লিখিত আকারে প্রসার ঘটিয়েছিলেন তিনি আক্কাদ শহর প্রতিষ্ঠিত করার প্রথমদিকে নিজেকে প্রায়শয়ই আক্কাদীয়ান রাজা হিসিবে প্রচার করতেন পরে তিনি কোন এক সময় কিস শহর অধিগ্রহণ করে নেন পরবর্তীতে মেসোপটেমিয়ার বৃহদাংশও দখল করে নেন এবং ক্রমে ক্রমে সরগন আক্কদীয়ান রাজা ইনান্নার তত্ত্বাবধায়ক কিস এর রাজা আনুর স্থলাভিষিক্ত রাজ্যেরমেসোপটেমিয়া রাজা এনলিলের রাজ্যপালএনসি নামে নিজেকে প্রচার করে ছিলেন। যদিও সুমেরিয়ান রাজাদের তালিকার অনেক অনুলিপিতে সারগনের শাসনকাল ৫৬ ৫৫ বা ৫৪ বছর বলা হয়েছে কিন্তু তা

In [51]:
print(len(context_val))

for i in range(len(context_val)):
  if question_val[i].find('?') == -1:
    question_val[i] = question_val[i] + '?'

for i in range(len(question_val)):
  if question_val[i].find(' ?') == -1:
    question_val[i] = question_val[i].replace('?', ' ?')

c = 0
for i in range(len(context_val)):
  if question_val[i].find(' ?') != -1:
    c = c+1
print(c)

1484
1484


In [52]:
sample_total = len(context_val)
print('No of sample in Validation set: ', sample_total)

f1 = 0.0
em = 0.0

with torch.no_grad():
    l = len(context_val)
    for i in range(l):
        pred = evaluate_one_text(question_val[i] + ' হ্যাঁ না । ' + context_val[i])
        pred = pred.replace('#', '')
        
        if(answer_type_val[i] == 'multiple spans'):
          f1 = f1 + get_updated_f1(pred, answer_val[i])
        else:
          f1 = f1 + compute_f1(pred, answer_val[i])
        em = em + compute_exact_match(pred, answer_val[i])

print('Validation set EM score: ', em/sample_total)
print('Validation set F1 score: ', f1/sample_total)

No of sample in Validation set:  1484
Validation set EM score:  0.2715633423180593
Validation set F1 score:  0.38940750435383736


### BanglaRQA Test

In [53]:
# loading the test dataset for calculating the EM and F1 scores on BanglaRQA's test set

import json
import os

f = open(os.path.join(base,test_file_name))
  
data_test = json.load(f)
  
data_test.keys()

data_test = data_test['data']

In [54]:
context_test = []
question_test = []
answer_test = []
question_type_test = []
answer_type_test = []

for i in range(len(data_test)):
    for j in range(len(data_test[i]['qas'])):
        context_test.append(remove_punc(normalize(data_test[i]['context'])))
        question_test.append(remove_punc(normalize(data_test[i]['qas'][j]['question_text'])))
        question_type_test.append(normalize(data_test[i]['qas'][j]['question_type']))
        answer_test.append(normalize(data_test[i]['qas'][j]['answers']['answer_text'][0]))
        answer_type_test.append(normalize(data_test[i]['qas'][j]['answers']['answer_type'][0]))

In [55]:
print(len(context_test))
print(context_test[0])

for i in range(len(context_test)):
  context_test[i] = context_test[i].replace('।', ' ।')
  answer_test[i] = answer_test[i].replace('।', ' ।')
  question_test[i] = question_test[i].replace('।', ' ।')

print(context_test[0])

1493
অক্ষয় তৃতীয়া হল চান্দ্র বৈশাখ মাসের শুক্লাতৃতীয়া অর্থাৎ শুক্লপক্ষের তৃতীয়া তিথি। হিন্দু ও জৈন ধর্মাবলম্বীদের কাছে এটি একটি বিশেষ তাৎপর্যপূর্ণ তিথি। এই শুভদিনে জন্ম নিয়েছিলেন বিষ্ণুর ষষ্ঠ অবতার পরশুরাম। বেদব্যাস ও গণেশ এই দিনে মহাভারত রচনা আরম্ভ করেন। এদিনই সত্য যুগ শেষ হয়ে ত্রেতাযুগের সূচনা হয়। এদিনই রাজা ভগীরথ গঙ্গা দেবীকে মর্ত্যে নিয়ে এসেছিলেন। এদিনই কুবেরের তপস্যায় তুষ্ট হয়ে মহাদেব তাকে অতুল ঐশ্বর্য প্রদান করেন। এদিনই কুবেরের লক্ষ্মী লাভ হয়েছিল বলে এদিন বৈভবলক্ষ্মীর পূজা করা হয়। এদিন থেকেই পুরীধামে জগন্নাথদেবের রথযাত্রা উপলক্ষে রথ নির্মাণ শুরু হয়। অক্ষয় শব্দের অর্থ হল যা ক্ষয়প্রাপ্ত হয় না। বৈদিক বিশ্বাসানুসারে এই পবিত্র তিথিতে কোন শুভকার্য সম্পন্ন হলে তা অনন্তকাল অক্ষয় হয়ে থাকে। কেদারবদ্রীগঙ্গোত্রীযমুনোত্রীর যে মন্দির ছয়মাস বন্ধ থাকে এইদিনেই তার দ্বার উদ্‌ঘাটন হয়। দ্বার খুললেই দেখা যায় সেই অক্ষয়দীপ যা ছয়মাস আগে জ্বালিয়ে আসা হয়েছিল।
অক্ষয় তৃতীয়া হল চান্দ্র বৈশাখ মাসের শুক্লাতৃতীয়া অর্থাৎ শুক্লপক্ষের তৃতীয়া তিথি । হিন্দু ও জৈন ধর্মাবলম্বীদের কাছে এটি 

In [56]:
print(len(context_test))

for i in range(len(context_test)):
  if question_test[i].find('?') == -1:
    question_test[i] = question_test[i] + '?'

for i in range(len(context_test)):
  if question_test[i].find(' ?') == -1:
    question_test[i] = question_test[i].replace('?', ' ?')

c = 0
for i in range(len(context_test)):
  if question_test[i].find(' ?') != -1:
    c = c+1
print(c)

1493
1493


In [57]:
anslysis = {
    'answerable_f1': 0.0,
    'answerable_em': 0.0,
    'answerable_cnt': 0.0,

    'unanswerable_f1': 0.0,
    'unanswerable_em': 0.0,
    'unanswerable_cnt': 0.0,

    'factoid_f1': 0.0,
    'factoid_em': 0.0,
    'factoid_cnt': 0.0,

    'causal_f1': 0.0,
    'causal_em': 0.0,
    'causal_cnt': 0.0,

    'confirmation_f1': 0.0,
    'confirmation_em': 0.0,
    'confirmation_cnt': 0.0,

    'list_f1': 0.0,
    'list_em': 0.0,
    'list_cnt': 0.0,
}

In [58]:
sample_total = len(context_test)
print('No of samples in Test Set: ',sample_total)

f1_total = 0.0
em_total = 0.0

with torch.no_grad():
    l = len(context_test)
    for i in range(l):
        pred = evaluate_one_text(question_test[i] + ' হ্যাঁ না । ' + context_test[i])
        pred = pred.replace('#', '')
        
        if(answer_type_test[i] == 'multiple spans'):
          f1 = get_updated_f1(pred, answer_test[i])
        else:
          f1 = compute_f1(pred, answer_test[i])
        f1_total = f1_total + f1
        em = compute_exact_match(pred, answer_test[i])
        em_total = em_total + em

        if(answer_test[i] == ''):
          anslysis['unanswerable_cnt'] = anslysis['unanswerable_cnt'] +1
          anslysis['unanswerable_f1'] = anslysis['unanswerable_f1'] + f1
          anslysis['unanswerable_em'] = anslysis['unanswerable_em'] + em
        else:
          anslysis['answerable_cnt'] = anslysis['answerable_cnt'] +1
          anslysis['answerable_f1'] = anslysis['answerable_f1'] + f1
          anslysis['answerable_em'] = anslysis['answerable_em'] + em
          
        if(question_type_test[i] == 'factoid'):
          anslysis['factoid_cnt'] = anslysis['factoid_cnt'] +1
          anslysis['factoid_f1'] = anslysis['factoid_f1'] + f1
          anslysis['factoid_em'] = anslysis['factoid_em'] + em
        elif (question_type_test[i] == 'causal'):
          anslysis['causal_cnt'] = anslysis['causal_cnt'] +1
          anslysis['causal_f1'] = anslysis['causal_f1'] + f1
          anslysis['causal_em'] = anslysis['causal_em'] + em
        elif (question_type_test[i] == 'confirmation'):
          anslysis['confirmation_cnt'] = anslysis['confirmation_cnt'] +1
          anslysis['confirmation_f1'] = anslysis['confirmation_f1'] + f1
          anslysis['confirmation_em'] = anslysis['confirmation_em'] + em
        elif (question_type_test[i] == 'list'):
          anslysis['list_cnt'] = anslysis['list_cnt'] +1
          anslysis['list_f1'] = anslysis['list_f1'] + f1
          anslysis['list_em'] = anslysis['list_em'] + em


print('----------------------------------------------------------------------------------------')
print()
print('Test set EM score: ', em_total/sample_total)
print('Test set F1 score: ', f1_total/sample_total)
print()
print('----------------------------------------------------------------------------------------')

print('Accuracy based on answerable/unanserable questions: ------------------------------------')
print()
print('Answerabale_em: ', anslysis['answerable_em']/anslysis['answerable_cnt'])
print('Answerabale_f1: ', anslysis['answerable_f1']/anslysis['answerable_cnt'])
print()
print('Unnswerabale_em: ', anslysis['unanswerable_em']/anslysis['unanswerable_cnt'])
print('Unnswerabale_f1: ', anslysis['unanswerable_f1']/anslysis['unanswerable_cnt'])
print()
print("----------------------------------------------------------------------------------------")

print('Accuracy based on different question type:----------------------------------------------')
print()
print('Factoid_em: ', anslysis['factoid_em']/anslysis['factoid_cnt'])
print('Facoid_f1: ', anslysis['factoid_f1']/anslysis['factoid_cnt'])
print()
print('causal_em: ', anslysis['causal_em']/anslysis['causal_cnt'])
print('causal_f1: ', anslysis['causal_f1']/anslysis['causal_cnt'])
print()
print('confirmation_em: ', anslysis['confirmation_em']/anslysis['confirmation_cnt'])
print('confirmation_f1: ', anslysis['confirmation_f1']/anslysis['confirmation_cnt'])
print()
print('list_em: ', anslysis['list_em']/anslysis['list_cnt'])
print('list_f1: ', anslysis['list_f1']/anslysis['list_cnt'])
print()
print('----------------------------------------------------------------------------------------')

No of samples in Test Set:  1493
----------------------------------------------------------------------------------------

Test set EM score:  0.2853315472203617
Test set F1 score:  0.394046228557564

----------------------------------------------------------------------------------------
Accuracy based on answerable/unanserable questions: ------------------------------------

Answerabale_em:  0.07473309608540925
Answerabale_f1:  0.21913791747014505

Unnswerabale_em:  0.926829268292683
Unnswerabale_f1:  0.926829268292683

----------------------------------------------------------------------------------------
Accuracy based on different question type:----------------------------------------------

Factoid_em:  0.31177606177606176
Facoid_f1:  0.4262545193221432

causal_em:  0.3375796178343949
causal_f1:  0.5279815105101408

confirmation_em:  0.1323529411764706
confirmation_f1:  0.1323529411764706

list_em:  0.1951219512195122
list_f1:  0.2793795126134812

-------------------------------

In [59]:
answer_anslysis = {
    'single_f1': 0.0,
    'single_em': 0.0,
    'single_cnt': 0.0,

    'multi_f1': 0.0,
    'multi_em': 0.0,
    'multi_cnt': 0.0,

    'yes_f1': 0.0,
    'yes_em': 0.0,
    'yes_cnt': 0.0,
}

In [60]:
sample_total = len(context_test)
print('No of samples in Test set: ', sample_total)

with torch.no_grad():
    l = len(context_test)
    for i in range(l):

        if(answer_test[i] != ''):  
          pred = evaluate_one_text(question_test[i] + ' হ্যাঁ না । ' + context_test[i])
          pred = pred.replace('#', '')

          if(answer_type_test[i] == 'multiple spans'):
            f1 = get_updated_f1(pred, answer_test[i])
          else:
            f1 = compute_f1(pred, answer_test[i])
          em = compute_exact_match(pred, answer_test[i])  
          
          if(answer_type_test[i] == 'single span'):
            answer_anslysis['single_cnt'] = answer_anslysis['single_cnt'] +1
            answer_anslysis['single_f1'] = answer_anslysis['single_f1'] + f1
            answer_anslysis['single_em'] = answer_anslysis['single_em'] + em
          elif (answer_type_test[i] == 'multiple spans'):
            answer_anslysis['multi_cnt'] = answer_anslysis['multi_cnt'] +1
            answer_anslysis['multi_f1'] = answer_anslysis['multi_f1'] + f1
            answer_anslysis['multi_em'] = answer_anslysis['multi_em'] + em
          elif (answer_type_test[i] == 'yes/no'):
            answer_anslysis['yes_cnt'] = answer_anslysis['yes_cnt'] +1
            answer_anslysis['yes_f1'] = answer_anslysis['yes_f1'] + f1
            answer_anslysis['yes_em'] = answer_anslysis['yes_em'] + em

print('Accuracy based on different answer type:-----------------------------------------------')
print()
print('single_span_em: ', answer_anslysis['single_em']/answer_anslysis['single_cnt'])
print('single_span_f1: ', answer_anslysis['single_f1']/answer_anslysis['single_cnt'])
print()
print('multiple_spans_em: ', answer_anslysis['multi_em']/answer_anslysis['multi_cnt'])
print('multiple_spans_f1: ', answer_anslysis['multi_f1']/answer_anslysis['multi_cnt'])
print()
print('yes_no_em: ', answer_anslysis['yes_em']/answer_anslysis['yes_cnt'])
print('yes_no_f1: ', answer_anslysis['yes_f1']/answer_anslysis['yes_cnt'])
print()
print('---------------------------------------------------------------------------------------')

No of samples in Test set:  1493
Accuracy based on different answer type:-----------------------------------------------

single_span_em:  0.09005847953216374
single_span_f1:  0.2612688649937155

multiple_spans_em:  0.045454545454545456
multiple_spans_f1:  0.1488710367975084

yes_no_em:  0.0
yes_no_f1:  0.0

---------------------------------------------------------------------------------------


### bn_squad Test

In [61]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [62]:
from datasets import load_dataset

In [63]:
test_dataset_squad = load_dataset("csebuetnlp/squad_bn", split="test")

In [64]:
test_context_squad = []
test_question_squad = []
test_answer_squad = []

for i in range(len(test_dataset_squad)):
    if(len(test_dataset_squad[i]['answers']['text']) != 0):
        test_context_squad.append(remove_punc(normalize(test_dataset_squad[i]['context'])))
        test_question_squad.append(remove_punc(normalize(test_dataset_squad[i]['question'])))
        test_answer_squad.append(normalize(test_dataset_squad[i]['answers']['text'][0]))
    else:
        test_context_squad.append(remove_punc(normalize(test_dataset_squad[i]['context'])))
        test_question_squad.append(remove_punc(normalize(test_dataset_squad[i]['question'])))
        test_answer_squad.append(normalize(''))

In [65]:
print(len(test_context_squad))
print(test_context_squad[0])

for i in range(len(test_context_squad)):
  test_context_squad[i] = test_context_squad[i].replace('।', ' ।')
  test_answer_squad[i] = test_answer_squad[i].replace('।', ' ।')
  test_question_squad[i] = test_question_squad[i].replace('।', ' ।')

print(test_context_squad[0])

2504
১০। উইমেন্স স্টাডিজ বিভাগ
১০ । উইমেন্স স্টাডিজ বিভাগ


In [66]:
print(len(test_context_squad))

for i in range(len(test_context_squad)):
  if test_question_squad[i].find('?') == -1:
    test_question_squad[i] = test_question_squad[i] + '?'

for i in range(len(test_context_squad)):
  if test_question_squad[i].find(' ?') == -1:
    test_question_squad[i] = test_question_squad[i].replace('?', ' ?')

c = 0
for i in range(len(test_context_squad)):
  if test_question_squad[i].find(' ?') != -1:
    c = c+1
print(c)

2504
2504


In [67]:
f1_total = 0.0
em_total = 0.0

sample_total = len(test_context_squad)
print(sample_total)

with torch.no_grad():
    l = len(test_context_squad)
    for i in range(l):
        pred = evaluate_one_text(test_question_squad[i] + ' হ্যাঁ না । ' + test_context_squad[i])
        pred = pred.replace('#', '')
          
        f1_total = f1_total + compute_f1(pred, test_answer_squad[i])
        em_total = em_total + compute_exact_match(pred, test_answer_squad[i])
        
        '''print(i)
        print(test_question_squad[i])
        print(pred)
        print(test_answer_squad[i])
        print()'''

          


print('Accuracy of our model on bn_quad dataset: ---------------------------------------------')
print()
print('EM score: ', em_total/sample_total)
print('F1 score: ', f1_total/sample_total)
print()
print('---------------------------------------------------------------------------------------')

2504
Accuracy of our model on bn_quad dataset: ---------------------------------------------

EM score:  0.505591054313099
F1 score:  0.535702129431574

---------------------------------------------------------------------------------------


In [68]:
with torch.no_grad():
    l = len(test_context_squad)
    for i in range(l):
        print(i)
        pred = evaluate_one_text(test_question_squad[i] + ' হ্যাঁ না । ' + test_context_squad[i])
        pred = pred.replace('#', '')
        if(i<20):
          print(test_question_squad[i])
          print(test_context_squad[i])
          print(pred)
          print(test_answer_squad[i])
        else:
          break
        

0
কেমব্রিজ বিশ্ববিদ্যালযয়ের প্রথম আচার্য কে ?
১০ । উইমেন্স স্টাডিজ বিভাগ


1
মহাকবি মাইকেল মধুসূদন দত্তের প্রথম কাব্যগ্রন্থটি কবে প্রকাশিত হয় ?
মধুসূদন দত্ত নাট্যকার হিসেবেই প্রথম বাংলা সাহিত্যের অঙ্গনে পদার্পণ করেন । রামনারায়ণ তর্করত্ন বিরচিত রত্নাবলী নাটকের ইংরেজি অনুবাদ করতে গিয়ে তিনি বাংলা নাট্যসাহিত্যে উপযুক্ত নাটকের অভাব বোধ করেন । এই অভাব পূরণের লক্ষ্য নিয়েই তিনি নাটক লেখায় আগ্রহী হয়েছিলেন । ১৮৫৯ খ্রিস্টাব্দে তিনি রচনা করেন শর্মিষ্ঠা নাটক । এটিই প্রকৃত অর্থে বাংলা ভাষায় রচিত প্রথম মৌলিক নাটক । ১৮৬০ খ্রিষ্টাব্দে তিনি রচনা করেন দুটি প্রহসন যথা একেই কি বলে সভ্যতা এবং বুড়ো শালিকের ঘাড়ে রোঁ এবং পূর্ণাঙ্গ পদ্মাবতী নাটক । পদ্মাবতী নাটকেই তিনি প্রথম অমিত্রাক্ষর ছন্দ ব্যবহার করেন । ১৮৬০ খ্রিষ্টাব্দে তিনি অমিত্রাক্ষরে লেখেন তিলোত্তমাসম্ভব কাব্য । এরপর একে একে রচিত হয় মেঘনাদ বধ কাব্য ১৮৬১ নামে মহাকাব্য ব্রজাঙ্গনা কাব্য ১৮৬১ কৃষ্ণকুমারী নাটক ১৮৬১ বীরাঙ্গনা কাব্য ১৮৬২ চতুর্দশপদী কবিতা ১৮৬৬ ।


2
শেখ মুজিবুর রহমান কবে জন্মগ্রহণ করেন ?
শেখ মুজিবুর রহমান ১৭ মার্চ ১৯২০  ১৫ আগস্ট ১৯৭৫ 